In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [30]:
os.environ['TOKEN'] = "AAAAAAAAAAAAAAAAAAAAADMkXQEAAAAAXbTfVvdItOoXToIeHsmgVNHE%2B64%3DAvYhn6Rulu9tgYemf91HyLMRkJKo5MmZC17Vg0FQDsUxO41gG8"

In [31]:
def auth():
    return os.getenv('TOKEN')

In [32]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [42]:
def create_url(user_id, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/1.1/friends/ids.json" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'user_id': user_id
                    }
    
#     'query': keyword,
#                     'start_time': start_date,
#                     'end_time': end_date,
#                     'max_results': max_results,
#                     'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
#                     'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
#                     'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
#                     'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
#                     'next_token': {}
    return (search_url, query_params)

In [39]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [48]:
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "xbox lang:en"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 15
user_id = "2540338484"

In [49]:
url = create_url(user_id, start_time,end_time, max_results)

In [50]:
json_response = connect_to_endpoint(url[0], headers, url[1])
json_response

Endpoint Response Code: 200


{'ids': [1135628532237754368,
  2209750495,
  294428111,
  1360894320,
  773683319309496320,
  1515221305,
  1313512883888680961,
  2350461168,
  2851572509,
  4041536238,
  1470944601309528072,
  60402615,
  151312222,
  1123624739774369793,
  51684922,
  4764590844,
  1969568858,
  1137056964259639296,
  21738639,
  85100906,
  1463714090631634953,
  274318389,
  929832230,
  795877410784813056,
  1057644712348303360,
  797933042564210694,
  1403538186727481350,
  1157431069215510531,
  1302175621536243713,
  14142150,
  1202532692723666946,
  1320481047361003521,
  1367259410398142470,
  950220903719170049,
  950597058330812417,
  267377585,
  1290773268392218626,
  904382812425396224,
  1453421875279892480,
  750354658628108288,
  1268363655130857473,
  143710939,
  345700384,
  635138557,
  361336287,
  766442747175854088,
  1071160775627161601,
  1245756170402316288,
  5940622,
  356589396,
  1214598690,
  20910115,
  19405852,
  2429413770,
  1408245052401405954,
  121426479,
  

In [47]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "ids": [
        1465728402372059144,
        1432782439944908801,
        848582669730369536,
        2812478524,
        1119859495,
        900040284616114176,
        1439623206424354820,
        1574841006,
        2986289819,
        52332354,
        232294292,
        3363584909,
        4922450361,
        94111955,
        352837463,
        1680458172,
        1217308137566412800,
        869703623688822784,
        840746981190447105,
        1058359603,
        6029522,
        926944612698206209,
        1311407216466030596,
        1353048395657719809,
        826350522,
        1389647947873280010,
        1863305378,
        19044790,
        328956435,
        1129851964139827200,
        2865007695,
        1384366692097171457,
        1015890234939269120,
        1703364902,
        140491831,
        1177300366620184576,
        3293760861,
        26000689,
        135356716,
        1156038692416491521,
        1199439010633961473,
        12623751654905405

In [54]:
len(json_response["ids"])

1265

In [55]:
df = pd.DataFrame(json_response)

,ids,next_cursor,next_cursor_str,previous_cursor,previous_cursor_str,total_count
0,1135628532237754368,0,0,0,0,None
1,2209750495,0,0,0,0,None
2,294428111,0,0,0,0,None
3,1360894320,0,0,0,0,None
4,773683319309496320,0,0,0,0,None
...,...,...,...,...,...,...
1260,107451380,0,0,0,0,None
1261,531697681,0,0,0,0,None
1262,415601652,0,0,0,0,None
1263,185123070,0,0,0,0,None


## Using Tweepy

In [58]:
import tweepy

In [81]:
import pygsheets
import pandas as pd
#authorization
gc = pygsheets.authorize(service_file='../../data/google/symmetric-sonar-834-d2afc313ab44.json')

In [82]:
gc.spreadsheet_titles()

['twitter_app_aouth']

In [86]:
sh = gc.open('twitter_app_aouth')

In [95]:
twitter_auth_df = sh.sheet1.get_as_df()

In [96]:
twitter_auth_df

,creator,name_of_app,consumer_key,consumer_secret,access_token,access_token_secret
0,Joe Hoover,jehgrabber,4V6baqjm6jF7fm0FUPtDoejor,UrS8vdHK7C6NAytN0i7rF0FFxuHfDrkpspFa4FapunvHz8...,462542884-clRFdPkAjb5po9GiybmWunDZgI4IEFHHidXw...,QspIv5dsk7gfRmyAFfPIGIvBnpnBZLpcwYa0yVuFqfTRX
1,Joe Hoover,scraper_bot1,Ep3sETt9T7ZVBLEYZ8r3QlGsR,r3nbKSaLCjkBOmNpfojOedPpqQ7E8SS1ZwKunmc3jQ6U5G...,462542884-nvG1dmfDK9yAteTOeVsZb53gOvZoF9pbcbhf...,EMua2XeEXGg9iaRRsfr79NEwtd2dV8Guw4KjDbZGQWheT
2,Joe Hoover,scraper_bot2,qnsUaMCxjQP2j1Y8Z7car4AqU,VonIKGkXT7XSO6cY82TvlBWeD40DzboXxxdZYfvfrSCFKr...,462542884-gZqNu8PhJuY4tS8nTvfmHOWPyNHk8EnyUUw4...,5GIZVCu0dFrjM2g18QdNlDYXSiE1tCXcdOKB6q6sdlAMs
3,Joe Hoover,scraper_bot3,PF3Xt3sziluK6fercG7cc08EU,jaeuaCh1YQSH7ByDNBlcMOePS3lHVX7mNCiYHXpZJi0RoV...,462542884-IU8jx4xKJjTKVApL3oyF31EhwDqQxJVqK57I...,fTHgpECg5dkoaEH5WTWm5abCiQeluGvhXx5g20Jcu39f5
4,Joe Hoover,scraper_bot4,mYd89lXH74xQKnVMXxVImX7tK,VEFn5VL6yuNImgWjPF4xJZLzMGYf7Vn5OOK0XuUXth5Mqi...,462542884-sSMs0G8txrGazlYFD2T5Ft0NqQSmD2soXnbN...,0I2n1TlFKnQemGm71KHUBbt7dgnJrGDMaJUfA0sNlxI9z
5,Joe Hoover,scraper_bot5,z9moOtldy4qV0AMuHlsm2KdbX,1gLEtLjAqjpydDQPy3QRmYlCcHJUyuNygDiC49ih69i7TE...,462542884-UezizTobWCHatz1zWnaJEGH3xz1DaaCdUHta...,pGjSh9X7RtW8wshpb0H6xZZAeiQDImNF5OTUxCQVSquHa
6,Joe Hoover,scraper_bot6,GjU4ZzQoJd628Ez4WOP1hhYhS,EYM3b7LdxxihXMTnCidf0eikd7EHBVERHcgp611MRNMvE0...,462542884-LwNsKzpAZWiJgiY5QAsgr4Taue4f6IbtQk3i...,HQ5gUYLnuo96k2xXIk8xCO25i5MUnHVWGtT15yyfotxSo
7,Joe Hoover,scraper_bot7,uXIPh1lb2HoTTVPkPYGCxWOxE,k0a115kSzW8Mf55OIWivIKJDzvAAn8u2zTl3J9la5tUyYr...,462542884-mVDQcvXkmydp046R57IHdNUBByR1zXnP7cxL...,WFCRFtHV1GNkERbvSQ3tAK0X6MGR4VdJfoQNQJjLIX1P9
8,Joe Hoover,scraper_bot8,gkXZTfngjc1UNNmuevAW12VTz,EMA2aIilp2X9lxaKlQczAEfEEhmo4LepiS5t8GytUluDEK...,462542884-b817WNO25e2Ybmzr0J76E8AJmrVDYpWVbRo2...,aQt89XSme8n3UTTnHZB5xH0pX3MSGozSrOxz1Nf1mKEmZ
9,Joe Hoover,scraper_bot9,ljaMPudQMLF0kubym33TgDMo6,v90GHMIW34IPx72RMOmwsj0mTLP4dwBZo7qfVoDuEwenyM...,462542884-BRHWYoZNIdznxEqrr2anrF4aNDtYWjupyFay...,dclewmEpRPzBQfP6e2J8RSnUHK4zRpSwBQlhf7JKtypMG


In [200]:

def get_twitter_auth(twitter_auth_df, counter):
    while True:
        counter = (counter+1)%len(twitter_auth_df)
        yield twitter_auth_df.iloc[counter]
    
    

In [201]:
auth = get_twitter_auth(twitter_auth_df, counter=0)

In [206]:
auth_info = next(auth)

In [208]:
tweepy_auth = tweepy.OAuthHandler(auth_info.consumer_key, auth_info.consumer_secret)
tweepy_auth.set_access_token(auth_info.access_token, auth_info.access_token_secret)

In [209]:
api = tweepy.API(tweepy_auth)

In [245]:
def renew_auth(auth_iterator):
    auth_info = next(auth_iterator)
    tweepy_auth = tweepy.OAuthHandler(auth_info.consumer_key, auth_info.consumer_secret)
    tweepy_auth.set_access_token(auth_info.access_token, auth_info.access_token_secret)
    api = tweepy.API(tweepy_auth)
    return api

In [256]:
def get_following(api, user_id, auth_iterator):
    print('Getting Following list of ',user_id)
    friends = []
    friends_screenName = []
    friend_ids = tweepy.Cursor(api.get_friend_ids, user_id=user_id).items()
    return friend_ids

     
    

In [273]:
from tqdm import tqdm
users = pd.read_csv("../../data/user_ids_5mil_7days.csv")
df_dict = {"user_id":[], "friends":[]}
user_count = 0
for user_id in tqdm(users.user_id):
    print(user_id)
    while True:
        try:
            friends = get_following(api, user_id, auth)
            df_dict["user_id"].append(user_id)
            df_dict["friends"].append(list(friends))
            user_count+=1
            if user_count%1000 == 1:
                df = pd.DataFrame(df_dict)
                df.to_csv("../../data/friends_python_res/friends.csv", index=False)
            break
        except Exception as e:
            print(e)
            api = renew_auth(auth)
            continue
        
    

  0%|          | 0/4683905 [00:00<?, ?it/s]

878069666429534208
Getting Following list of  878069666429534208
400 Bad Request
215 - Bad Authentication data.
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 T

429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list 

429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
401 Unauthorized
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401

429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
401 Unauthorized
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  87806966642953420

401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Followin

401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
401 Unauthorized
89 - Invalid or expired token.
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
401 Authorization Required
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
429 Too Many Requests
88 - Rate limit exceeded
Getting Following list of  878069666429534208
400 Bad Request
215 - Bad Authentication data.
Getting Following list of  8780696664295

  0%|          | 0/4683905 [00:36<?, ?it/s]


KeyboardInterrupt: 

In [270]:
list(friends)

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

878069666429534208
769092703007154177
529470770
911417378323308545
88207285
1426840441
889421930863906816
842454411729285120
344090136
39212905
463479104
1282158128
1282158128
922820338072686592
2193686280
356179022
928642764
3711232468
1162639716
871858796179673089
1192870717
902707770289414144
2882200121
38647512
797551581436723200
7964842
603531050
14063398
39538375
721453089945993216
859768982483685377
2842169794
251869750
15815676
845369406544924674
191162512
1282158128
30168446
259839167
113897359
742707630871851010
1282158128
3623357233
278415655
844808520687992833
29558150
1171793868
740590915379564544
23228167
63806853
967789195
90324779
2378985289
3431874803
720927815206641664
604940737
165488177
231431466
1337102588
21026718
3227979529
789936132234489856
742707630871851010
801284108676567040
186258648
4121124994
742707630871851010
2749687734
1167735445
730681858627936256
772595763004059648
17582910
866981923091906560
914667436850286592
903556257356816384
14715685
10441742
23

176075043
74082962
815581400338288641
128537084
152051108
885367753993056256
2483462785
851464401362100224
17514726
761114737740439552
340005956
2207827233
1271852322
211301250
5548282
132828087
3301528211
824521282863591425
2884845544
1289704663
3258209034
559394390
23228167
37301624
808176556455710720
366414568
16582697
904160286860562432
780580183614959616
35607021
3049806133
909757470532411392
2369887456
94375629
2780005678
925643100369940480
2577487746
95333464
889341722634854405
1276583298
14188041
4724195840
30345957
919868257963728896
2799330871
520142210
3092043930
2305447849
269383362
2315982086
826481877250883584
318880714
2493963250
44166215
92772806
270213356
254818045
260670614
787699681962946560
634470391
782264299238617088
279329101
925351243102474241
3289407552
459799742
16365255
2531525700
130888746
755113
865666170611007489
3225820305
34878722
544828911
4737370153
32156060
4825516313
820607108576083969
1571404508
752826505
874590102
249054237
799498589772857344
75373

72552866
70700679
521115135
4774058968
837334176479215619
57439579
96747014
91120560
91120560
4786581027
828387782976425984
555826466
799484072439513088
717210018
904165600603275269
210569909
171463972
2375415205
19420667
3385464521
900437108363591682
1443799376
808697508125937664
505322398
761126037153611776
778756321784979456
813369488997421056
18488262
860831029698932739
835224839929675776
595823977
38400004
121499083
39120304
3441990557
37705479
84335372
3153848415
48586050
110899077
979793040
1133821555
898723165978767360
98263693
1397757626
2315499817
800129647891529728
178335736
820612988717535236
2330424144
24463149
3257481188
131223451
372548499
790173608178028544
864078511111274496
239177242
888379708114698240
3933716057
848634008699273216
774864242
3401375327
26802202
246072365
571120922
51207072
532611579
901597280603648001
50731581
1352736158
2311231
734703365800792064
362197517
466864852
2422083498
2591516127
174141309
917688634228203520
31762979
752077173914214400
238702

24719945
821587004727132160
1553031854
807231608168255490
278237218
727995007517507585
1372563925
928398811
2871410219
3317756286
146256240
3010587036
59005491
1182075564
99913258
20204298
16250929
167095342
1050151435
377582164
185167847
2985673373
50811808
820759428140036096
1064452536
139408851
819416797354954752
735890817949138944
320880647
18364588
3273057716
4658845539
262485769
844253288967081984
17659340
863885504567181312
27971734
817347063444865024
757055083683209216
578353910
316534109
336952539
13953112
144691619
22881120
1162639716
167254347
1455933535
2802522952
571241678
890909185076076545
4452031515
21618140
3244681899
2192406870
900027903005609986
309380449
2534173706
832012825191030784
3652661776
4686424633
1295911188
829388481398910976
29976145
289733605
37521277
72919841
855899761026662402
127925965
362534018
724558864272658433
27808121
218376044
801534350558892032
846985336618147840
1135701193
31282804
2255627316
4705764139
22280154
822871173444747264
408895203
889

954395618
631540231
2184303804
526132574
3060462260
1002920520
1020799974
1700224028
524955355
20722768
881759886
3313054675
3230198245
69750777
379949906
2243353709
117365017
886026002
30995845
20721841
179298078
575410217
3365000232
827191308930396160
14860628
316810246
2567979293
372502604
207731256
55909861
529470770
17819416
65109514
239522450
62936742
1518350994
2562556158
2254871191
45349751
55909861
38388750
449171780
844183688
336668326
71189823
2830890790
340825844
17292912
991599836
589349701
782058822080737280
398310670
321423359
518090185
517091068
1564775696
32976913
16754136
913382213579767809
862307986609512448
277155475
775802865378029568
2874845313
109354790
906592843577397249
903628194
2803085292
113231300
1187814913
1010261293
846430088
760655915674501120
555748320
3177894841
1566886314
2275787358
524963688
1000404620
525987302
1564699868
807247949356998657
855274374
1868795131
754913
848536095801446406
905399728224878593
35890299
348600754
16946467
182792075
325477

...

616532645
2817403699
20268039
560703845
2890895885
110695390
120385561
21539403
53981431
859036802
1645876892
2885150428
890495622423433217
122154024
347437629
1517458362
870465760459718657
388084214
50025191
973980152
3308254063
890229317871046656
1466312012
10207502
32789881
29657049
2836774399
363630971
3475416197
1162639716
999771745
725077790240833540
3120013892
2987465340
3363037205
3637992076
58302354
536669718
98993077
822444348977463296
83055581
1360903585
313758408
824632513
37499827
21313364
2735511486
1161161875
22144947
902295275267649536
2279872633
198368615
802667088959705090
887383877647556613
16365255
599355701
724027759819776000
34507278
17061111
457639568
801967126525444096
760283851650322433
36739057
2573590074
706111979992305664
897231852242477056
754007629496582145
590382121
50925084
363997208
262487815
2724215808
49964679
870035354509815813
42933980
265722834
1537798038
207649522
1247957906
14808307
1475149910
779430801482125312
519706573
412745445
71708140113750

2879075602
185355059
153083123
755058196423794688
3508357753
3145998633
61309849
124318369
19456704
831232031334027264
2315964782
2713942316
344314746
126028024
296359418
47359542
17753712
23488196
305792802
491196570
485959164
396875760
476995113
1465083278
15979262
779263625609510912
188751970
85457517
166986355
364087338
330925718
38284349
76105892
1950455654
271277936
352012951
21986898
1926278622
2351885644
22529401
2737116474
89302374
269783513
637348179
196784042
171790562
797959540666957826
828970771502530560
590201048
747886830
19234875
2488541
391761599
3293575002
920638727784927232
11744152
613027315
1136684586
480594888
1116011700
482411748
25467533
111051026
293768205
111583614
324636914
134259008
19063323
253316481
97132065
19567593
453055027
1959551
51632731
2376468298
4173460000
91475817
220643866
41573220
480843927
612895209
743949780322230272
54438283
26849343
166168776
40532427
41626659
41465485
824410682
1227846332
16917401
915274959768997888
123058126
284205584
621

750405708525473792
31066684
359885557
3950816657
367094939
86141342
3052966202
156536241
85650541
329808478
32084765
36511031
14516541
404973133
618572539
5736542
225647847
1942444554
52230962
35121351
604709916
190721927
435098362
805202785
27616308
404660197
38253034
763750096412389378
159592026
3422773625
529063492
2270376266
44931391
417860017
597755959
22360892
488072140
271447589
1482175074
1127087880
222310198
898630153043161088
67653725
257215286
3011795599
15907183
3367924167
499230786
738607986164174849
201039266
2713346152
883184340586422272
241339183
2985058060
2405332538
380456273
138847737
3300194365
3254294504
2632680430
1246744686
2269311254
885957619919200256
345728801
763883419818336256
430994382
96415687
894268118125727746
15125592
460246990
21333520
516065423
2600829318
25183606
46912960
88230087
23501344
76534742
2237798034
153111721
828265635218063363
70886494
16026110
4898977128
1901394096
16657965
1345616456
773078778935377920
55075546
35629382
16052008
16051958

1153304688
2714798737
4403170402
585946129
3095224204
948880980
2270588034
457831111
631663116
2942962205
3742685722
15926388
546087883
126050164
119802433
150970255
56142895
783786017606213633
848306430633426944
893237365
4687840987
165880475
113865008
824069085721952256
382828137
816387353233227776
246755579
260926419
90345954
90946310
913023860710297600
46433782
864958783
28944830
25728193
20587567
98069039
900093468357402630
829442212719063040
3084630964
1969511725
18945689
497364817
1045733126
90250196
870923887
632191966
24767346
733481546812821507
247291350
18706621
59117898
772625238026694656
1319315437
15859242
302098529
3631865176
44617334
889989663972352000
1311174307
14332159
382637564
25907240
15181842
2427573765
100310841
234810773
2807393883
35929206
2518057904
2996037676
880572480800292864
724703956853731330
3135346139
4201205487
747464891766759424
3765696437
832220757035884545
1671883176
156212667
527095730
135310881
575410217
74054498
2672800712
24401045
177239135
773

KeyboardInterrupt: 